In [97]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
from random import randint
import pandas as pd

In [190]:
driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver')
driver.get('https://www.realestate.moj.gov.kw/live/Moj_Rs_11.aspx')

/tmp/ipykernel_5410/521642558.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver')


### Helper Function

In [103]:
def small_sleep(low=1, high=10):
    sleep(randint(low,high))

In [104]:
def large_sleep(low=1, high=20):
    sleep(randint(low,high))

In [66]:
def get_data(driver):
    data = {}
    elements = driver.find_elements(By.XPATH("//*"))
    for element in elements:
        if element.get_attribute("id"):
            data[element.get_attribute("id")] = element.get_attribute("value")
    return data

In [ ]:
def check_button(driver, id):
    element = driver.find_element_by_id(id)
    if not element.is_selected():
        element.click()

In [94]:
def click_button(driver, id):
    element = driver.find_element(By.ID(id))
    element.click()

In [ ]:
def get_html(driver):
    html = driver.page_source
    return html

In [32]:
def get_beautiful_soup(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup


In [38]:
def get_table_data(driver):
    source = get_beautiful_soup(driver)
    table = source.find('table')
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    return True

### Testing

In [ ]:
driver.find_element

In [ ]:
inputx.click()

In [ ]:
jave = "javascript:__doPostBack('ctl00$bodyContent$gridList','Page$2')"

## main Form

In [191]:
from_date = '5/1/2023'
to_date = "23/01/2023"

In [192]:
from_date_el = driver.find_element(By.ID, "bodyContent_fromDate")
from_date_el.send_keys(from_date)

In [193]:
to_date_el = driver.find_element(By.ID, "bodyContent_tillDate")
to_date_el.send_keys(to_date)

In [194]:
## click to main button
checked = False

In [195]:
if checked== False:
    driver.execute_script('document.getElementById("bodyContent_cbGov").click()')
    small_sleep()
    driver.execute_script('document.getElementById("bodyContent_cbZone").click()')
    small_sleep()
    driver.execute_script('document.getElementById("bodyContent_cbCat").click()')
    small_sleep()
    checked = True

In [196]:
if checked==True:
    driver.execute_script('document.getElementById("bodyContent_btnSubmit").click()')

## get table data

In [43]:
source = get_beautiful_soup(driver)

In [34]:
data = get_table_data(driver)

## pages

In [185]:
def get_first_page(driver=driver):
    return int(driver.find_elements(By.TAG_NAME, 'a')[0].get_attribute('href').split(sep=',')[-1].split(sep="$")[-1][:-2])

In [198]:
def get_last_page(driver=driver):
    return int(driver.find_elements(By.TAG_NAME, 'a')[-1].get_attribute('href').split(sep=',')[-1].split(sep="$")[-1][:-2])

In [115]:
href = driver.find_elements(By.TAG_NAME, 'a')[9].get_attribute('href')

In [118]:
href.split(sep=',')[-1]

"'Page$11')"

In [151]:
last_page = get_last_page(driver)

In [152]:
last_page

11

In [173]:
driver.find_elements(By.TAG_NAME, 'a')[0].get_attribute('href')

"javascript:__doPostBack('ctl00$bodyContent$gridList','Page$1')"

In [153]:
first_page = get_first_page()
first_page 

2

In [169]:
first_page=9

In [42]:
page_numbers = len(driver.find_elements(By.TAG_NAME, 'a'))

In [ ]:
driver.execute_script(f"document.getElementsByTagName('a')[{last_page}].click()")
large_sleep()

In [172]:
 driver.execute_script(f"document.getElementsByTagName('a')[{first_page}].click()")

In [199]:
last_page = get_last_page()

In [219]:
def check_reach_end():
    last_page_temp = get_last_page()
    if last_page == last_page_temp:
        return True
    return False

In [220]:
check_reach_end()

True

In [216]:
work = True

In [217]:
i = 0
while work:
    if check_reach_end() == False:
        driver.execute_script(f"document.getElementsByTagName('a')[{i}].click()")
        i += 1
        large_sleep()
    else:
        work = False

In [203]:
i

4

In [210]:
driver.execute_script(f"document.getElementsByTagName('a')[3].click()")

### Main loop

In [40]:
data = []

In [43]:
for page_number in range(page_numbers):
    driver.execute_script(f"document.getElementsByTagName('a')[{page_number}].click()")
    large_sleep()
    get_table_data(driver)

### convert to dataframe

In [48]:
columns = ["المحافظة","المنطقة",
          "فئة العقار",
          "وصف العقار",
          "لتصرف",
                    "النسبة",
          "القطعة",
          "التاريخ",
                    "المساحة",
          "سعر البيع","none1",'none']

In [49]:
df = pd.DataFrame(data=data,columns=columns)
df.head()

,المحافظة,المنطقة,فئة العقار,وصف العقار,لتصرف,النسبة,القطعة,التاريخ,المساحة,سعر البيع,none1,none
0,None,None,None,None,None,None,None,None,None,None,None,None
1,مبارك الكبير,المسايل,خاص,بيت,بيع,%100.000,5,05/01/2023,400,730000,None,None
2,حولي,حولي,إستثماري,ارض,بيع,%100.000,43,05/01/2023,762,1245000,None,None
3,الفروانية,الفروانية,إستثماري,شقة,بيع,%100.000,0,05/01/2023,68.27,57000,None,None
4,الأحمدي,التوسعة الشمالية بالفنطاس,إستثماري,شقة,بيع,%100.000,0,05/01/2023,208.59,135000,None,None


In [51]:
 df.shape

(330, 12)

In [55]:
df.drop(labels=["none","none1"], axis=1, inplace=True)

In [56]:
df.to_csv("data.csv")

In [58]:
df.tail()

,المحافظة,المنطقة,فئة العقار,وصف العقار,لتصرف,النسبة,القطعة,التاريخ,المساحة,سعر البيع
325,الأحمدي,الصباحية,خاص,بيت,بيع,%100.000,3,05/12/2022,375,245000
326,حولي,ض.الصديق,خاص,أرض,بيع,%100.000,7,05/12/2022,400,400000
327,الفروانية,غرب عبدالله المبارك,خاص,بيت,بيع,%100.000,5,05/12/2022,400,485000
328,...45678910111213,...,4,5,6,7,8,9,10,11
329,...,4,5,6,7,8,9,10,11,12
